This notebook performs linear regression on our honey production and air quality combined dataset, which is read in in the second cell<br>
We use the matrix solution to minimize the MSE<br>
Cross validation is performed and an MSE is output near the end. Everything is written from scratch with numpy

In [13]:
import pandas as pd
from scipy.stats import norm
import numpy as np
import math

In [14]:
data = pd.read_csv("data/completeFeatureVectors.csv")

X = data[['o3','co','so2','no2','pm25_frm', 'pressure', 'temperature', 'wind', 'year']].to_numpy()
# subtract 1998 from the year so that it starts at zero
X[:,8] = X[:,8]-1998
# Append ones to the start of X for the bias term
X = np.append(np.ones((X.shape[0],1)), X, axis=1)
y = data[['yield_per_col']].to_numpy()

Minimize the MSE by using the matrix formula to find theta.<br>
Also find the variance sigma so that we can find confidence intervals for our predictions

In [15]:
np.set_printoptions(suppress=True) # disable scientific notation when printing

theta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)),X.T),y)

# print out theta with labels
for label,theta_i in zip(['intercept','o3','co','so2','no2','pm25_frm','pressure','temperature','wind','year'], theta):
    print(label + ": " + "{:f}".format(theta_i[0]))

intercept: 120.190591
o3: -544.230626
co: -20.476653
so2: 0.998524
no2: 0.152943
pm25_frm: -0.686405
pressure: 0.002974
temperature: 0.317392
wind: -0.375663
year: -1.848450


Make a prediction and show a 95% confidence interval for that prediction

In [16]:
x = np.array([[1],[0.0312],[0.759],[1.575],[14.757],[43.9],[984.],[59.17],[97.27], [10]])
prediction = np.matmul(x.T, theta)[0][0]
print("prediction:", prediction)

# find variance sigma and find a 95% confidence interval
sigma_sq = (np.matmul((y-np.matmul(X, theta)).T,(y-np.matmul(X, theta))) / X.shape[0]-1)[0][0]
sigma = math.sqrt(sigma_sq)
dev = sigma_sq*np.matmul(np.matmul(x.T,np.linalg.inv(np.matmul(X.T,X))),x)[0][0]
print("dev:", dev) # getting 190... seems very high, not sure how to confirm
ppf = norm.ppf(0.975, loc=0, scale=dev) # how ppf works: https://stackoverflow.com/questions/60699836/how-to-use-norm-ppf
print("ppf:", ppf)
rng = ppf*sigma # ppf returns a standard deviation multiplier, so we need to multiply by the stdev to get the true value

print("range:", rng)
print("95% conf. Lower bound:", prediction-rng)
print("95% conf. Upper bound:", prediction+rng)

prediction: 28.046243398850937
dev: 190.05662274680785
ppf: 372.50413560705937
range: 5076.42310982046
95% conf. Lower bound: -5048.3768664216095
95% conf. Upper bound: 5104.4693532193105


Determine which features are significant using the inverse tail of chi squared function

In [17]:
cov_matrix = sigma_sq*np.linalg.inv(np.matmul(X.T,X))
threshold = 0.2357 # inverse tail of chi squared(?) function
features = ['intercept','o3','co','so2','no2','pm25_frm','pressure','temperature','wind','year']
for i in range(theta.shape[0]):
    sig = (theta[i,:][0] / cov_matrix[i,i])**2
    if sig > threshold:
        print(str(i) + ": " + features[i] + ": significant")
    else:
        print(str(i) + ": " + features[i] + ": not significant")

0: intercept: significant
1: o3: not significant
2: co: significant
3: so2: significant
4: no2: significant
5: pm25_frm: significant
6: pressure: significant
7: temperature: significant
8: wind: significant
9: year: significant


Perform k-fold cross validation to find our mean squared error (MSE)

In [18]:
k = 10

# the size of the testing set for each fold
chunk_size = X.shape[0] // k

# shuffle X and y together
Xy_shuffled = np.append(X, y, axis=1)
np.random.shuffle(Xy_shuffled)

sq_errors = []
abs_errors = []

# iterate through k folds
for i in range(k):

    # split out testing and training data
    X_k_test = Xy_shuffled[chunk_size*i:chunk_size*(i+1),:10]
    y_k_test = Xy_shuffled[chunk_size*i:chunk_size*(i+1),10]

    if i == 0:
        X_k_train = Xy_shuffled[chunk_size:,:10]
        y_k_train = Xy_shuffled[chunk_size:,10]
    elif i == k-1:
        X_k_train = Xy_shuffled[:chunk_size*i,:10]
        y_k_train = Xy_shuffled[:chunk_size*i,10]
    else:
        X_k_train = np.append(Xy_shuffled[:chunk_size*i,:10], Xy_shuffled[chunk_size*(i+1):,:10], axis=0)
        y_k_train = np.append(Xy_shuffled[:chunk_size*i,10], Xy_shuffled[chunk_size*(i+1):,10], axis=0)

    # train on training data to get theta_k
    theta_k = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_k_train.T, X_k_train)),X_k_train.T),y_k_train)
    # test out the theta_k that we found on the testing chunk
    for i in range(X_k_test.shape[0]):
        y_pred = np.matmul(X_k_test[i,:].T, theta_k)
        y_actual = y_k_test[i]
        #print("predicted:", y_pred, "actual", y_actual)
        sq_errors.append((y_pred - y_actual)**2)
        abs_errors.append(abs(y_pred-y_actual))
    
mean_sq_error = np.mean(sq_errors)
mean_abs_error = np.mean(abs_errors)
print("Mean squared error:", mean_sq_error)
print("Mean absolute error:", mean_abs_error)

Mean squared error: 208.7902477777134
Mean absolute error: 11.246244252785345
